# Homework 05 - Topic modeling

_Goal_ :

**We want to run topic modeling over the corpus to "discover" the main topics of the emails.**

_Tools_ :

**The tools used are :**

* pandas
* [gensim](https://radimrehurek.com/gensim/index.html)

_Contents_ :

* [1 - Loading data](#1---Loading-data)
* [2 - Topic modeling](#2---Topic-modeling)
* [3 - Tweaking the number of topics](#3---Tweaking-the-number-of-topics)

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from gensim import corpora
from gensim.models.ldamodel import LdaModel

# For DataFrame pretty-printing
from IPython.display import display

%matplotlib inline
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

/Users/maximepeschard/.pyenv/versions/3.5.2/Python.framework/Versions/3.5/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


# 1 - Loading data

In the first place, as usual, we load the `Emails.csv` file with pandas to form a DataFrame.

In [2]:
emails = pd.read_csv('hillary-clinton-emails/Emails.csv')
emails.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


# 2 - Topic modeling

Here, we use `ExtractedBodyText` as our raw corpus

In [3]:
raw_texts = emails.ExtractedBodyText.dropna()
raw_texts[:5]

1    B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...
2                                                  Thx
4    H <hrod17@clintonemail.com>\nFriday, March 11,...
5    Pis print.\n-•-...-^\nH < hrod17@clintonernail...
7    H <hrod17@clintonemail.corn>\nFriday, March 11...
Name: ExtractedBodyText, dtype: object

Since this data is raw, we first have to pre-process it so that our results reflect meaningful topics. To do so we simply apply the same steps as in the first assignment (tokenization, stopwords removal and stemming).

In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

tokens = [nltk.word_tokenize(text.lower()) for text in raw_texts]
stop_words = set(stopwords.words('english'))

# This part is a bit empirical and domain specific
punctuation = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '-', '--', '...', '•', "''", '""', '``', '@', '<', '>', "'s"]
commons = ['pm','re','fw','via','fyi',"n't","'ll","'m",'ok','thx','in','the','no','yes','said','sent','mr.','call','meet','say','get','can','would','could','meeting','office']
# We make the choice to remove any "word" containing digit(s) for topic modeling
def contains_digits(s):
    return any(c.isdigit() for c in s)

stop_words.update(punctuation)
stop_words.update(commons)
filtered_tokens = [[t for t in tok if (t not in stop_words and len(t)>2 and not contains_digits(t))] for tok in tokens]

ps = PorterStemmer()
texts = [[ps.stem(t) for t in tok] for tok in filtered_tokens]

In [5]:
texts[13]

['hope', 'see', 'pictur', 'kamala']

The previous command shows that some emails are probably very short and will not necessarily help for topic modeling. That's why we decide to exclude emails that contains less than a fixed number of words.

*NB : Another possiblity could be to consider **threads** (ie. emails and replies) as documents instead of single emails. We do **not** follow this approach here since it is rather time consuming to link emails together.*

In [6]:
print("Before removing short emails :", len(texts), "emails")

threshold = 5
long_texts = [text for text in texts if len(text) > threshold]

print("After removing short emails :", len(long_texts), "emails")

Before removing short emails : 6742 emails
After removing short emails : 3775 emails


Now we can build a dictionary containing all the words in our raw corpus. Then, we build the actual corpus that will be used to do topic modeling : it is represented as a list of lists (one per document), and each inner list contains words ids (wrt. the previously built dictionary) and corresponding number of occurences in the document.

In [7]:
dictionary = corpora.Dictionary(long_texts)
corpus = [dictionary.doc2bow(text) for text in long_texts]

Before running LDA, let's just define a helper function to print the results in a nicer way.

In [8]:
# Helper function to print topics in a nicer way
def print_lda_topics(lda_model, numtopics, with_probabilities=True):
    for topic_id, topic_words in lda_model.show_topics(num_topics=numtopics, formatted=False):
        words = list(map(lambda x: x[0], topic_words))
        probabilities = list(map(lambda x: round(x[1],4), topic_words))
        if with_probabilities:
            print("Topic", topic_id, ':')
            display(pd.DataFrame([probabilities], columns=words))
        else:
            print("Topic", topic_id, end=': ')
            print(' | '.join(words))

Finally, we can run LDA and print the resulting topics. We choose to focus on 5 topics for this first example.

In [9]:
lda = LdaModel(corpus, num_topics=5, id2word=dictionary)
print_lda_topics(lda, 5)

Topic 0 :


,secretari,state,depart,room,arriv,time,rout,privat,resid,hous
0,0.0218,0.0185,0.0163,0.01,0.0076,0.0075,0.0068,0.0062,0.0059,0.0055


Topic 1 :


,work,state,one,want,peopl,state.gov,govern,u.s.,also,need
0,0.0058,0.0057,0.0043,0.004,0.0039,0.0038,0.0034,0.0034,0.0034,0.0031


Topic 2 :


,like,presid,also,time,state,vote,govern,work,new,need
0,0.0044,0.0041,0.004,0.0039,0.0038,0.0037,0.0036,0.0036,0.0035,0.0033


Topic 3 :


,obama,one,parti,presid,time,polit,year,republican,american,democrat
0,0.0053,0.0051,0.0045,0.0042,0.0042,0.004,0.004,0.0037,0.0037,0.0035


Topic 4 :


,obama,new,american,state,presid,republican,one,senat,issu,democrat
0,0.0071,0.0049,0.0046,0.0044,0.0044,0.0044,0.0043,0.0037,0.0036,0.0035


From this first example, we cannot really distinguish and label topics at first sight. In the following section, we try different parameters for the number of topics and discuss the results.

# 3 - Tweaking the number of topics

Here, we try to explore a bit the results with different number of topics for LDA. For the sake of readability, we don't print the probabilities.

In [10]:
num_topics = [10, 20, 30]

for n in num_topics:
    # Print parameter choice
    print("\t\tnum_topics =",n)
    
    # Run LDA
    lda = LdaModel(corpus, num_topics=n, id2word=dictionary)
    # Print results
    print_lda_topics(lda, n, with_probabilities=False)
    
    # Print separator
    print("===============================================")

		num_topics = 10
Topic 0: state | depart | work | secretari | u.s. | new | case | hous | time | one
Topic 1: time | one | state | polit | american | need | presid | israel | work | like
Topic 2: secretari | state | depart | time | obama | room | presid | new | hous | privat
Topic 3: work | women | know | state | one | help | u.s. | like | discuss | need
Topic 4: israel | american | state | peac | govern | one | new | obama | also | isra
Topic 5: state | senat | secretari | depart | clinton | assist | presid | work | time | nation
Topic 6: one | obama | state | presid | boehner | work | may | parti | peopl | nation
Topic 7: vote | like | democrat | republican | obama | one | state | presid | work | american
Topic 8: want | also | obama | afghanistan | mcchrystal | work | see | think | one | new
Topic 9: parti | state.gov | huma | presid | abedin | work | time | last | govern | first
		num_topics = 20
Topic 0: obama | senat | presid | republican | clinton | state | democrat | time | one

**Discussion on the results :**

Again, due to Porter stemming, this is a sometimes a bit hard to read. Still, we can make some observations :
* when the number of topics is small, we cannot really attribute a "label" to each one, they mostly share words
* when the number increases, it becomes easier to distinguish some topics : for example with `num_topics = 30`, *Topic 8* is clearly about Israeli–Palestinian conflict, *Topic 9* is about Ireland and *Topic 20* seems to be about votes/elections.

However, in most cases, a lot of topics seem irrelevant. Maybe a better pre-processing would help, since here we use a pretty basic pipeline.